## 充分理解name和variable_scope

###  1.1 问题: 首先当我们定义一个神经网络的时候,很多时候网络的参数很多,这个时候网络可能显示出杂乱无章

### 1.2 为了解决这个问题，我们引入了 name_scope 和 variable_scope， 二者又分别承担着不同的责任：

* name_scope: * 为了更好地管理变量的命名空间而提出的。比如在 tensorboard 中，因为引入了 name_scope， 我们的 Graph 看起来才井然有序。
* variable_scope: * 大大大部分情况下，跟 tf.get_variable() 配合使用，实现变量共享的功能

### 2.1 三种方式定义变量区别 
* tf.placeholder()
* tf.Variable()
* tf.get_variable()

In [9]:
# 1 placeholder
import tensorflow as tf
v1 = tf.placeholder(tf.float32, shape=[2, 3, 4])
print(v1.name)
v1 = tf.placeholder(tf.float32, shape=[2, 3, 4],name='ph')
print(v1.name)
v1 = tf.placeholder(tf.float32, shape=[2, 3, 4],name='ph')
print(v1.name)
print(type(v1))
print(v1)

Placeholder_7:0
ph_12:0
ph_13:0
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("ph_13:0", shape=(2, 3, 4), dtype=float32)


In [15]:
# 2 tf.Variable()
v2 = tf.Variable([1,2],dtype=tf.float32)
print(v2.name)
v2 = tf.Variable([1,2],dtype=tf.float32,name='V')
print(v2.name)
v2 = tf.Variable([1,2],dtype=tf.float32,name='x')
print(v2.name)
print(type(v2))
print(v2)

Variable_1:0
V_2:0
x:0
<class 'tensorflow.python.ops.variables.Variable'>
<tf.Variable 'x:0' shape=(2,) dtype=float32_ref>


In [4]:
# 3. tf.get_variable() 创建变量的时候需要提供name
import tensorflow as tf
v3 = tf.get_variable(name='gv',shape=[])
print(v3.name)
v4 = tf.get_variable(name='gf',shape=[])
print(v4.name)

## 反正就是显示一大堆错误

#### 2.2 从上面的实验结果来看，这三种方式所定义的变量具有相同的类型。而且只有 tf.get_variable() 创建的变量之间会发生命名冲突。在实
际使用中，三种创建变量方式的用途也是分工非常明确的。其中

* tf.placeholder() 占位符。* trainable==False *
* tf.Variable() 一般变量用这种方式定义。 * 可以选择 trainable 类型 *
* tf.get_variable() 一般都是和 tf.variable_scope() 配合使用，从而实现变量共享的功能。 * 可以选择 trainable 类型 *

### 3.1  熟悉name_scope和variable_scope区别


In [1]:
import tensorflow as tf
with tf.name_scope('nsc1'):
    v1 = tf.Variable([1], name='v1')
    with tf.variable_scope('vsc1'):
        v2 = tf.Variable([1], name='v2')
        v3 = tf.get_variable(name='v3', shape=[])
print('v1.name: '+ v1.name)
print('v2.name: ' + v2.name)
print ('v3.name: ' + v3.name)

# v1.name:  nsc1/v1:0
# v2.name:  nsc1/vsc1/v2:0
# v3.name:  vsc1/v3:0
with tf.name_scope('nsc1'):
    v4 = tf.Variable([1], name='v4')
print('v4.name: ' + v4.name)
# v4.name:  nsc1_1/v4:0

In [8]:
import tensorflow as tf
with tf.variable_scope("foo"):
      v = tf.get_variable("v1", [1])
with tf.variable_scope("foo", reuse=True):  #表示使用了共享变量了
      v1 = tf.get_variable("v1", [1])
assert v1 == v